## PIPELINES PROJECT - Jacobo Pintos

La base de datos está bajada de esta web de Kaggle: https://www.kaggle.com/essonnel/5000-imdb-movies-multivariant-analysis

In [28]:
import pandas as pd
import os
import requests
import quandl
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from dotenv import load_dotenv
from collections import Counter

In [29]:
DataFrame='movie_metadata.csv'

### 1. Read function

Creo una función para leer el dataset. Esta función obtendrá el dataset de la dirección que pongamos en la casilla anterior "DataFrame".

In [51]:
def read(DataFrame):
    imdb = pd.read_csv(DataFrame)
    return imdb

read(DataFrame).head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


### 2. Set up function

Selecciono aquellas columnas que considero importantes para mi análisis. 

Cabe destacar que no he seleccionado las columnas con información de actores por pelicula debido a que en un vistazo rápido se ha visto que faltan datos importantes sobre esta columna en el dataset.

In [31]:
def set_up(imdb):
    colnames = ['movie_title', 'country', 'title_year', 'duration', 'genres',
                'budget', 'gross', 'director_name' ,'imdb_score' , 'num_voted_users']
    imdb = pd.DataFrame(imdb, columns=colnames)
    return imdb

### 3. Cleaning function

Esta fase presenta los siguientes procesos:

- Ordeno los valores por valoración del público para una mejor visualización. 
- Limpio los posibles elementos duplicados y los valores nulos del dataset. 
- Cambio la tipología de algunas columnas para convertirlas a enteros y redondeo otras para dejar solo un elemento decimal en el float.
- Calculo máximos y minimos para ver cual sería un mínimo de votación aceptable para incluír una película y lo sitúo en este minimo más de 5000 votos, por lo que elimino todas aquellas con la función drop que tengan menos de 5000 votos.
- Por último realizo un drop para eliminar aquellas películas que hayan tenido un presupuesto y unos beneficios menores a 100k US$ y se hayan producido fuera de USA.

In [32]:
def data_cleaning(imdb):
    imdb = imdb.sort_values(by='imdb_score', ascending=False)
    imdb = imdb.drop_duplicates(subset=['movie_title', 'title_year']).dropna()
    imdb['duration'] = imdb['duration'].astype(int)
    imdb['title_year'] = imdb['title_year'].astype(int)
    imdb['budget'] = imdb['budget'].round(1)
    imdb['gross'] = imdb['gross'].round(1)
    imdb['num_voted_users'] = imdb['num_voted_users'].round(1)
    imdb = imdb.drop(imdb[imdb['num_voted_users']<5000].index)
    imdb = imdb.drop(imdb[imdb['budget']<100000].index)
    imdb = imdb.drop(imdb[imdb['gross']<100000].index)
    imdb = imdb.drop(imdb[imdb['country']!='USA'].index)
    return imdb

### 4. Data wrangling

En esta fase:

- Convierto a Mn tanto las columnas de budget como de gross y las renombro.
- Paso a miles la volumna de votos de usuario y la renombro.
- Elimino las columnas antiguas que ya no hacen falta al crear las nuevas.
- Creo una columna de multiplicador de retorno de beneficio de cada película basado en las columnas de gross revenue y budget.


In [33]:
def data_wrangling(imdb):
    imdb['Budget (US$ Mn)'] = (imdb['budget']/(10**6))
    imdb['Gross Revenue (US$ Mn)'] = (imdb['gross']/(10**6))
    imdb['Total votes (k)'] = (imdb['num_voted_users']/(10**3))
    imdb.drop(['gross', 'budget', 'num_voted_users'], axis='columns', inplace=True)
    imdb['Gross Profit (x)'] = (imdb['Gross Revenue (US$ Mn)']/imdb['Budget (US$ Mn)'])
    imdb['Gross Profit (x)'] = imdb['Gross Profit (x)'].round(2)
    return imdb

### 5. Data binning

Clasificamos las columnas en bins dependiendo de su duración. En esta clasificacion tenemos:
- Peliculas con duración menor a 90 minutos.
- Peliculas con duración entre 90 minutos y dos horas.
- Peliculas con duración entre dos horas y dos horas y media.
- Peliculas de duración mayor a dos horas y media.

In [34]:
def data_binning(imdb):
    bins=[0, 90, 120, 150, 500]
    group_names=['Less than 90 min', 'Between 90 and 120 min', 
             'Between 120 and 150 min', 'More than 150 min']
    imdb['Duration range']=pd.cut(imdb['duration'], bins, labels=group_names)
    imdb = imdb.drop(columns=['duration'])
    return imdb

### 6. Data embellish

Ordenamos finalmente las columnas y les asignamos nombres finales de cara a su presentación en gráficas. Como posteriormente llamaremos a la API algunos de estos campos los tendremos luego que volver a modificar para que finalmente quede el DataFrame con el aspecto deseado.

In [35]:
def data_embellish(imdb):
    column_order = ['movie_title', 'title_year', 'country', 'genres', 'Duration range',
                    'director_name', 'imdb_score', 'Total votes (k)', 'Budget (US$ Mn)',
                   'Gross Revenue (US$ Mn)', 'Gross Profit (x)']
    imdb = imdb[column_order]
    imdb.rename(columns={'movie_title':'Title'}, inplace=True)
    imdb.rename(columns={'title_year':'Year'}, inplace=True)
    imdb.rename(columns={'country':'Country'}, inplace=True)
    imdb.rename(columns={'genres':'Genre'}, inplace=True)
    imdb.rename(columns={'director_name':'Director'}, inplace=True)
    imdb.rename(columns={'imdb_score':'Score'}, inplace=True)
    return imdb

### 7. API call

Llamamos a la API para conseguir el "Consumer Price Index" y así poder poner todos los Budget a precios de 2019.

El link de la API es: https://www.quandl.com/data/RATEINF/CPI_USA-Consumer-Price-Index-USA

Obtenemos el CPI trimestral desde 1917 cuya base 100 se situa entre 1982 y 1984. Procesaremos estos datos para convertirlos en media anual y a su vez actualizaremos su base a 1 en el año 2019 para así actualizar a US$ de 2019 simplemente dividiendo entre la columna que queramos actualizar.

In [5]:
token=open(".env").read()

def API_input(token):
    CPI_index = quandl.get("RATEINF/CPI_USA", authtoken=token)
    CPI_index.head()
    CPI_index = CPI_index.resample('Y').mean()
    CPI_index['Date YYYY'] = CPI_index.index
    CPI_index = CPI_index.reset_index()
    CPI_index['Year Date'] = CPI_index['Date YYYY'].map(lambda x: x.year)
    colnames=['Year Date', 'Value']
    CPI_index = pd.DataFrame(CPI_index, columns=colnames)
    CPI_index['Value'] = CPI_index['Value']/CPI_index['Value'][106]
    return CPI_index

### 8. API merge and final cleaning

Combinamos el dataframe obtenido de la API con el dataset IMDB para así ya poder aplicar el CPI a las columnas de budget y gross profit y tener estos datos normalizados a 2019.

In [37]:
def merge(imdb, CPI_index):
    imdb = imdb.merge(CPI_index, left_on='Year', right_on='Year Date')
    imdb['Budget (US$ Mn)'] = imdb['Budget (US$ Mn)']/imdb['Value']
    imdb['Gross Profit (x)'] = imdb['Gross Profit (x)']/imdb['Value']
    imdb['Budget (US$ Mn)'] = imdb['Budget (US$ Mn)'].round(2)
    imdb['Gross Profit (x)'] = imdb['Gross Profit (x)'].round(2)
    imdb['Gross Revenue (US$ Mn)'] = imdb['Gross Revenue (US$ Mn)'].round(2)
    imdb['Total votes (k)'] = imdb['Total votes (k)'].round(1)
    imdb['Gross Profit (x)'] = (imdb['Gross Revenue (US$ Mn)']/imdb['Budget (US$ Mn)'])
    imdb['Gross Profit (x)'] = imdb['Gross Profit (x)'].round(2)
    imdb = imdb.drop(columns=['Value'])
    imdb = imdb.drop(columns=['Year Date'])
    imdb = imdb.sort_values(by='Score', ascending=False)
    return imdb

### 9. Save

Defino una función para guardar el archivo y poder tener en un csv los datos limpios.

In [53]:
def toCSV(imdb):
    imdb.to_csv('Dataset procesado.csv')

In [54]:
run = read(DataFrame)
CPI_index = API_input(token)
imdb = set_up(run)
imdb = data_cleaning(imdb)
imdb = data_wrangling(imdb)
imdb = data_binning(imdb)
imdb = data_embellish(imdb)
imdb = merge(imdb, CPI_index)

imdb.head()

,Title,Year,Country,Genre,Duration range,Director,Score,Total votes (k),Budget (US$ Mn),Gross Revenue (US$ Mn),Gross Profit (x)
0,The Shawshank Redemption,1994,USA,Crime|Drama,Between 120 and 150 min,Frank Darabont,9.3,1689.8,42.65,28.34,0.66
40,The Godfather,1972,USA,Crime|Drama,More than 150 min,Francis Ford Coppola,9.2,1155.8,36.29,134.82,3.72
171,The Godfather: Part II,1974,USA,Crime|Drama,More than 150 min,Francis Ford Coppola,9.0,790.9,66.68,57.30,0.86
41,The Dark Knight,2008,USA,Action|Crime|Drama|Thriller,More than 150 min,Christopher Nolan,9.0,1676.2,217.30,533.32,2.45
174,The Lord of the Rings: The Return of the King,2003,USA,Action|Adventure|Drama|Fantasy,More than 150 min,Peter Jackson,8.9,1215.7,129.23,377.02,2.92


### 10. Plots

Esta función nos devolverá en formato de gráfica la cantidad total de películas que tienen una duración determinada en porcentaje. Esta gráfica así como las siguientes se mostrarán y se guardaran en la carpeta donde tenemos el archivo.

In [39]:
def plot_genre(imdb):
    category_data= imdb.Genre.str.split('|', expand=True).stack().value_counts(0)/len(imdb)*100
    category_data = category_data.round(1)
    plot_genre=category_data.head(5).plot('bar') 
    plt.title("Total movies by genre in %")
    plt.savefig('Plot by genres')


Esta función nos devolverá la cantidad total de películas que tienen una duración determinada. 

In [40]:
def plot_bins(imdb):
    v = imdb['Duration range'].value_counts()
    v.plot('bar')
    plt.title("Movie duration bins")
    plt.savefig('Plot by movies duration')


Esta función nos devolverá una gráfica con la cantidad de peliculas que tenemos en el dataframe por año. 

In [41]:
def plot_yearVSmovies(imdb):
    imdb2=imdb.drop(imdb[imdb['Year']<1970].index)
    imdb2=imdb2.drop(imdb[imdb['Year']>2015].index)
    plot_yearVSmovies = imdb2.groupby('Year').count()['Title'].plot(xticks = np.arange(1970,2016,5))

    sns.set(rc={'figure.figsize':(10,5)})
    plt.title("Year Vs Number of Movies",fontsize = 14)
    plt.xlabel('Release year',fontsize = 13)
    plt.ylabel('Number of Movies',fontsize = 13)
    plt.ylim(0,140)

    sns.set_style("whitegrid")
    plt.savefig('Relation movies by year')


Esta función nos devolverá una gráfica con la media de votos por pelicula por años.

In [42]:
def plot_yearVSvotes(imdb):
    imdb.groupby('Year').mean()['Total votes (k)'].plot(xticks = np.arange(1940,2016,5))

    sns.set(rc={'figure.figsize':(10,5)})
    plt.title("Year Vs Average # of votes",fontsize = 14)
    plt.xlabel('Release year',fontsize = 13)
    plt.ylabel('# of votes',fontsize = 13)

    plot_yearVSvotes = sns.set_style("whitegrid")
    plt.savefig('Relation years and mean votes')


Esta función nos devolverá las 10 películas con mayor presupuesto. 

In [43]:
def plot_top10budget(imdb):
    info = pd.DataFrame(imdb['Budget (US$ Mn)'].sort_values(ascending = False))
    info['Title'] = imdb['Title']
    data = list(map(str,(info['Title'])))

    x = list(data[:10])
    y = list(info['Budget (US$ Mn)'][:10])

    ax = sns.pointplot(x=y,y=x)
    sns.set(rc={'figure.figsize':(10,5)})
    ax.set_title("Top 10 High Budget Movies",fontsize = 15)
    ax.set_xlabel("Budget (US$ Mn)",fontsize = 13)
    plot_top10budget = sns.set_style("whitegrid")
    plt.savefig('Top 10 movies by budget')


Esta función nos devolverá las 10 películas con menor valoración. 

In [44]:
def plot_worsttop10(imdb):
    info = pd.DataFrame(imdb['Score'].sort_values(ascending = True))
    info['Title'] = imdb['Title']
    data = list(map(str,(info['Title'])))

    x = list(data[:10])
    y = list(info['Score'][:10])

    ax = sns.pointplot(x=y,y=x)
    sns.set(rc={'figure.figsize':(10,5)})
    ax.set_title("Top 10 less voted movies",fontsize = 15)
    ax.set_xlabel("Score",fontsize = 13)
    plot_top10budget = sns.set_style("whitegrid")
    plt.savefig('Top 10 less voted movies')


Esta función nos devolverá la tabla con la correlación entre la valoración de una pelicula y el presupuesto invertido en ella. 

In [45]:
def plot_corr_coreVSbudget(imdb):
    ax = sns.regplot(x=imdb['Score'], y=imdb['Budget (US$ Mn)'],color='c')

    ax.set_title("Score Vs Budget correlation",fontsize=13)
    ax.set_xlabel("Score",fontsize=12)
    ax.set_ylabel("Budget",fontsize=12)

    sns.set(rc={'figure.figsize':(6,4)})
    sns.set_style("whitegrid")

    imdb['Budget (US$ Mn)'] =imdb['Budget (US$ Mn)'].replace(0,np.NAN)
    imdb['Score'] =imdb['Score'].replace(0,np.NAN)

    data_corr = imdb.corr()
    print("Correlation Between Score And Budget : ",data_corr.loc['Score','Budget (US$ Mn)'])
    plt.savefig('Correlation between budget and score')


### Categories

Mediante las siguientes dos funciones obtenemos:

- En la primera llamada categorizer un ranking de directores según un parametro a definir entre valoración media de sus películas (Score), número total de títulos en el DataFrame (Title), presupuesto medio de sus películas (Budget) y profit medio de sus peliculas en base a un multiplicador obtenido entre dividir el total de revenue de sus peliculas entre el presupuesto.
- En la función categorizer obtenermos una serie de datos del director que introduzcamos por pantalla en formato tabla  detallado por pelicula (valoración de la pelicula, nombre, presupuesto gastado y multiplicador de beneficio).

Cabe destacar en el ranking que para poder visualizar la información lo más correctamente posible solo se admiten directores con un número de peliculas mayor a cinco para poder eliminar a estos directores que hayan triunfado con su opera prima pero luego no hayan continuado una carrera en el cine.

In [46]:
def categorizer(imdb):
    print("Selecciona una categoría para ordenar los directores. \n Categorías: Score, Title, Budget (US$ Mn), Gross Profit (x)")
    categoria = input()

    imdb_directors = imdb.groupby('Director', as_index=False).agg({"Score": "mean", 'Title':'count', 'Budget (US$ Mn)':'mean', 'Gross Profit (x)':'mean'})
    imdb_directors = imdb_directors.drop(imdb_directors[imdb_directors['Title']<5].index)
    imdb_directors = imdb_directors.sort_values(by='{}'.format(categoria), ascending=False)
    imdb_directors = imdb_directors.round(1)
    output1 = imdb_directors.head(10)
    return output1

In [47]:
def director(imdb):
    print("Selecciona una director para conocer su filmografía.")
    director = input()
    output2 = imdb.loc[imdb['Director']=='{}'.format(director),['Score', 'Title', 'Budget (US$ Mn)', 'Gross Profit (x)']]
    return output2